<a href="https://colab.research.google.com/github/itimes-digital/deep-learning-estudo/blob/main/serie_temporal_multi_saidas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
base = pd.read_csv('https://raw.githubusercontent.com/itimes-digital/deep-learning-estudo/main/dataset/petr4_treinamento.csv')
base.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,19.990000,20.209999,19.690001,19.690001,18.086271,30182600.0
1,2013-01-03,19.809999,20.400000,19.700001,20.400000,18.738441,30552600.0
2,2013-01-04,20.330000,20.620001,20.170000,20.430000,18.766001,36141000.0
3,2013-01-07,20.480000,20.670000,19.950001,20.080000,18.444506,28069600.0
4,2013-01-08,20.110001,20.230000,19.459999,19.500000,17.911745,29091300.0


In [4]:
base = base.dropna()

In [7]:
base_treinamento = base.iloc[:, 1:2].values
base_treinamento

array([[19.99    ],
       [19.809999],
       [20.33    ],
       ...,
       [15.99    ],
       [16.1     ],
       [16.1     ]])

In [8]:
base_valor_maximo = base.iloc[:, 2:3].values
base_valor_maximo

array([[20.209999],
       [20.4     ],
       [20.620001],
       ...,
       [16.139999],
       [16.129999],
       [16.1     ]])

In [11]:
normalizador = MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)
base_valor_maximo_normalizada = normalizador.fit_transform(base_valor_maximo)

In [13]:
previsores = []
preco_real_1 = []
preco_real_2 = []

for i in range(90, 1242):
  previsores.append(base_treinamento_normalizada[i - 90: i, 0]);
  preco_real_1.append(base_treinamento_normalizada[i, 0]);
  preco_real_2.append(base_valor_maximo_normalizada[i, 0]);

previsores, preco_real_1, preco_real_2 = np.array(previsores), np.array(preco_real_1), np.array(preco_real_2);

In [14]:
previsores.shape, preco_real_1.shape, preco_real_2.shape

((1152, 90), (1152,), (1152,))

In [15]:
previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1));

In [16]:
previsores.shape

(1152, 90, 1)

In [17]:
preco_real = np.column_stack((preco_real_1, preco_real_2))
preco_real.shape

(1152, 2)

In [21]:
model = Sequential();

model.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores.shape[1], 1)));
model.add(Dropout(0.3));

model.add(LSTM(units = 100, return_sequences = True));
model.add(Dropout(0.3));

model.add(LSTM(units = 50, return_sequences = True));
model.add(Dropout(0.3));

model.add(LSTM(units = 50));
model.add(Dropout(0.3));

model.add(Dense(units = 2, activation = 'linear'));

model.compile(optimizer = 'rmsprop', 
              loss = 'mean_squared_error', 
              metrics = ['mean_absolute_error']);

model.summary();

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 90, 100)           40800     
_________________________________________________________________
dropout_8 (Dropout)          (None, 90, 100)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 90, 100)           80400     
_________________________________________________________________
dropout_9 (Dropout)          (None, 90, 100)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 90, 50)            30200     
_________________________________________________________________
dropout_10 (Dropout)         (None, 90, 50)            0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 50)               

In [ ]:
model.fit(previsores, preco_real, epochs = 100, batch_size = 32)